In [126]:
import pandas as pd
import plotly 
import numpy as np
import plotly.figure_factory as ff
import scipy
#pip install scipy

In [127]:
# Import data

data = pd.read_csv('chocolate.csv')
continent = pd.read_csv('countryContinent.csv',encoding = "ISO-8859-1")
imp_exp=pd.read_csv('UNdata_Export_20220301_151116452.csv')
coord=pd.read_csv('country_points.csv',encoding = "ISO-8859-1")

In [128]:
coord.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [129]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [130]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [131]:
imp_exp=imp_exp[imp_exp['Commodity']=='Cocoa beans, whole or broken, raw or roasted'][['Country or Area','Year','Commodity','Flow','Quantity','Trade (USD)']]
imp_exp.head()

,Country or Area,Year,Commodity,Flow,Quantity,Trade (USD)
60,Albania,2012,"Cocoa beans, whole or broken, raw or roasted",Import,130.0,874.0
71,Albania,2011,"Cocoa beans, whole or broken, raw or roasted",Import,4524.0,19877.0
83,Albania,2010,"Cocoa beans, whole or broken, raw or roasted",Import,45.0,295.0
94,Albania,2009,"Cocoa beans, whole or broken, raw or roasted",Import,262.0,1919.0
149,Albania,2004,"Cocoa beans, whole or broken, raw or roasted",Import,1606.0,2729.0


In [132]:
# Prepar the data for the merge
data["company_location"] = data["company_location"].str.title()
data["country_of_bean_origin"] = data["country_of_bean_origin"].str.title()
data["company_location"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)'}, inplace=True)
data["country_of_bean_origin"].replace({'U.S.A': 'United States of America','U.K.':'United Kingdom of Great Britain and Northern Ireland','Dominican republic':'Dominican Republic','El salvador':'El Salvador','Vietnam':'Viet Nam','Venezuela':'Venezuela (Bolivarian Republic of)','South Korea':'Korea (Republic of)','New Zealand':'New Zealand','Russia':'Russian Federation','Taiwan':'Taiwan, Province of China','Sao Tome':'Sao Tome and Principe','Sao Tome & Principe':'Sao Tome and Principe','St. Lucia':'Saint Lucia','U.A.E.':'United Arab Emirates','St.Vincent-Grenadines':'Saint Vincent and the Grenadines','Bolivia':'Bolivia (Plurinational State of)','Burma':'Myanmar','Tanzania':'Tanzania, United Republic of','Trinidad':'Trinidad and Tobago','Dr Congo':'Congo (Democratic Republic of the)'}, inplace=True)

In [133]:
# removing Unnamed:0
data=data.iloc[:,1:]

In [134]:
continent.head()

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,150.0,154.0
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,2.0,15.0
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,9.0,61.0


In [135]:
teste=data.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'company_continent','sub_region':'company_region','code_2':'company_code_2'}), left_on='company_location', right_on='country', how='left')

In [136]:
teste[teste['company_continent'].isna()]['company_location'].value_counts

<bound method IndexOpsMixin.value_counts of 193        Wales
486     Scotland
487     Scotland
488     Scotland
489     Scotland
490     Scotland
491     Scotland
1083    Scotland
1084    Scotland
1513    Scotland
1514    Scotland
Name: company_location, dtype: object>

## Excluimos Scotland visto que faz parte do UK?

In [137]:

teste=teste.merge(continent[['country','continent','sub_region','code_2']].rename(columns={'continent':'bean_continent','sub_region':'bean_region','code_2':'bean_code_2'}), left_on='country_of_bean_origin', right_on='country', how='left')

In [138]:
teste[teste['bean_continent'].isna()]['country_of_bean_origin'].value_counts

<bound method IndexOpsMixin.value_counts of 88      Blend
92      Blend
96      Blend
100     Blend
177     Blend
        ...  
2198    Blend
2211    Blend
2213    Blend
2219    Blend
2223    Blend
Name: country_of_bean_origin, Length: 177, dtype: object>

In [139]:
teste.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,third_taste,fourth_taste,country_x,company_continent,company_region,company_code_2,country_y,bean_continent,bean_region,bean_code_2
0,2454,5150,United States of America,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,...,full body,NaN,United States of America,Americas,Northern America,US,Madagascar,Africa,Eastern Africa,MG
1,2458,5150,United States of America,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,...,savory,NaN,United States of America,Americas,Northern America,US,Dominican Republic,Americas,Caribbean,DO
2,2454,5150,United States of America,2019,"Tanzania, United Republic of","Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,...,bready,NaN,United States of America,Americas,Northern America,US,"Tanzania, United Republic of",Africa,Eastern Africa,TZ
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,...,roasty,NaN,France,Europe,Western Europe,FR,Peru,Americas,South America,PE
4,797,A. Morin,France,2012,Bolivia (Plurinational State of),Bolivia,70.0,3.50,4,have_bean,...,NaN,NaN,France,Europe,Western Europe,FR,Bolivia (Plurinational State of),Americas,South America,BO


In [140]:
data=teste

In [141]:
data.head()

,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,beans,...,third_taste,fourth_taste,country_x,company_continent,company_region,company_code_2,country_y,bean_continent,bean_region,bean_code_2
0,2454,5150,United States of America,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,have_bean,...,full body,NaN,United States of America,Americas,Northern America,US,Madagascar,Africa,Eastern Africa,MG
1,2458,5150,United States of America,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.50,3,have_bean,...,savory,NaN,United States of America,Americas,Northern America,US,Dominican Republic,Americas,Caribbean,DO
2,2454,5150,United States of America,2019,"Tanzania, United Republic of","Kokoa Kamili, batch 1",76.0,3.25,3,have_bean,...,bready,NaN,United States of America,Americas,Northern America,US,"Tanzania, United Republic of",Africa,Eastern Africa,TZ
3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,have_bean,...,roasty,NaN,France,Europe,Western Europe,FR,Peru,Americas,South America,PE
4,797,A. Morin,France,2012,Bolivia (Plurinational State of),Bolivia,70.0,3.50,4,have_bean,...,NaN,NaN,France,Europe,Western Europe,FR,Bolivia (Plurinational State of),Americas,South America,BO


In [142]:
# Add the coordinates both for bean_origin and company_location
coord.head()


,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [143]:
teste1=data.merge(coord.iloc[:,:-1], left_on='bean_code_2', right_on='country', how='left').rename(columns={'latitude':'bean_lat','longitude':'bean_lon'})
teste2=data.merge(coord.iloc[:,:-1], left_on='company_code_2', right_on='country', how='left').rename(columns={'latitude':'country_lat','longitude':'country_lon'})


In [144]:
var1_2=np.asarray(data[data['rating']<=2]['cocoa_percent'],dtype=np.float32)

var3=np.asarray(data[(data['rating']<=3) & (data['rating']>2) ]['cocoa_percent'],dtype=np.float32)
var4=np.asarray(data[(data['rating']<=4)&(data['rating']>3)]['cocoa_percent'],dtype=np.float32)

In [158]:
var0=np.asarray(data[data.rating <= data.rating.quantile(.25)]['cocoa_percent'],dtype=np.float32)
var1_2=np.asarray(data[(data.rating > data.rating.quantile(.25))&(data.rating <= data.rating.quantile(.50))]['cocoa_percent'],dtype=np.float32)
var3=np.asarray(data[(data.rating > data.rating.quantile(.50))&(data.rating <= data.rating.quantile(.75))]['cocoa_percent'],dtype=np.float32)
var4=np.asarray(data[data.rating > data.rating.quantile(.75)]['cocoa_percent'],dtype=np.float32)

In [160]:
# falta meter titulo xaxis yaxis
labels = ['kjg','Rating 1 and 2','Rating 3','Rating 4']
variables=[var0,var1_2,var3,var4]
fig = ff.create_distplot(variables,labels, show_hist=False)
fig.show()


In [149]:
len(var1_2)
len(var3)
len(var4)

1249